In [9]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [10]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [11]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [14]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
8,model_7_9_3,0.998650,0.978666,0.999315,0.998613,0.001494,0.001583,0.001017,0.001316,"Hidden Size=[25, 15], regularizer=0.01, learni..."
12,model_7_9_2,0.998513,0.985773,0.999413,0.998979,0.001646,0.001056,0.000872,0.000969,"Hidden Size=[25, 15], regularizer=0.01, learni..."
17,model_7_9_1,0.998310,0.991971,0.999497,0.999297,0.001871,0.000596,0.000747,0.000667,"Hidden Size=[25, 15], regularizer=0.01, learni..."
23,model_14_9_13,0.998223,0.993891,0.996318,0.995000,0.001966,0.006429,0.003636,0.005114,"Hidden Size=[25, 10], regularizer=0.1, learnin..."
25,model_14_9_12,0.998207,0.994155,0.996559,0.995253,0.001985,0.006151,0.003398,0.004855,"Hidden Size=[25, 10], regularizer=0.1, learnin..."
29,model_14_9_11,0.998175,0.994439,0.996818,0.995525,0.002020,0.005853,0.003142,0.004577,"Hidden Size=[25, 10], regularizer=0.1, learnin..."
35,model_14_9_10,0.998123,0.994741,0.997094,0.995815,0.002077,0.005535,0.002869,0.004280,"Hidden Size=[25, 10], regularizer=0.1, learnin..."
37,model_7_8_9,0.998092,0.995537,0.987922,0.995175,0.002111,0.006851,0.003496,0.005272,"Hidden Size=[25, 15], regularizer=0.01, learni..."
39,model_7_8_8,0.998085,0.995884,0.988544,0.995510,0.002120,0.006318,0.003316,0.004905,"Hidden Size=[25, 15], regularizer=0.01, learni..."
43,model_7_8_7,0.998055,0.996249,0.989151,0.995857,0.002152,0.005758,0.003140,0.004526,"Hidden Size=[25, 15], regularizer=0.01, learni..."
